# HW 1 - Carlos Alvarado

### Problem 1 - CODE

In [3]:
import geopandas as gpd
import os
import pandas as pd
import requests
import datetime

def get_and_load_data(data_info):
    '''
    Loads "data_name.csv" from local disk, or downloads it if it's not present
    
    Input: dictionary with information about required data
    Returns: pandas data_frame for "data_name"
    '''
    
    filepath = './data/{}.csv'.format(data_info['source'])
    
    if os.path.exists(filepath):
        data = pd.read_csv(filepath)
    else:
        data = helper_download_data(filepath, data_info['source'], data_info['datevar'])
    
    for old_var, new_var in data_info['rename']:
        data.rename(index=str, columns={old_var: new_var}, inplace=True)
        
    return data
    
def helper_download_data(filepath, data_name, datevar):
    offset = 0
    limit = 50000
    data = pd.DataFrame({})
    last_year = (datetime.datetime.now() - datetime.timedelta(days=365)).strftime('%Y-%m-%d')

    while True:
        
        url = 'https://data.cityofchicago.org/resource/' + \
              "{}.json?$order=service_request_number DESC &$limit={}&$offset={}&$where= {} > '{}'".format(
              data_name, limit, offset, datevar, last_year)
        print('getting data from', url)
    
        response = requests.get(url)
        response_code = response.status_code
        
        if response_code != 200: 
            print('Failed to download data')
            return data
            
        json_data = response.content

        if len(json_data) > 4:
            data = pd.concat([data, pd.read_json(json_data)])
        
            offset = offset + limit
        else:
            print(json_data)
            break
            
    #save data to csv for future use
    data.to_csv(filepath)
    
    return data    

In [4]:
# Loading complaints data. Added community names.

DATA1 = {'name': 'Graffiti Removal',
         'source': 'hec5-y4x5',
         'datevar': 'creation_date',
         'rename': [('where_is_the_graffiti_located_', 'Sub Type')]    
        }

DATA2 = {'name': 'Vacant and Abandoned Buildings Reported',
         'source': '7nii-7srd',
         'datevar': 'date_service_request_was_received',
         'rename': [('is_building_open_or_boarded_', 'Sub Type'), 
                    ('date_service_request_was_received', 'creation_date')]
        }

DATA3 = {'name': 'Pot Holes Reported',
         'source': '7as2-ds3y',
         'datevar': 'creation_date',
         'rename': [('zip', 'zip_code'), 
                    ('type_of_service_request', 'Sub Type')]
        }
    
DATA4 = {'name': 'Sanitation Code Complaints',
         'source': 'me59-5fac',
         'datevar': 'creation_date',         
         'rename': [('what_is_the_nature_of_this_code_violation_', 'Sub Type')]
        }

SOURCES = [DATA1, DATA2, DATA3, DATA4]

complaints = pd.DataFrame({})

for db_data in SOURCES:
    data = get_and_load_data(db_data)
    if not data.empty:
        print(data.columns)
        data['Complaint Type'] = db_data['name']
        complaints = pd.concat([complaints, data], ignore_index=True)

area_names_file = "./data/boundaries/Boundaries - Community Areas (current).geojson"
area_names = gpd.read_file(area_names_file)
area_names['area_numbe'] = pd.to_numeric(area_names.area_numbe, errors='coerce')

complaints = complaints.merge(area_names, left_on='community_area', right_on='area_numbe', how='left')

Index(['Unnamed: 0', 'community_area', 'completion_date', 'creation_date',
       'latitude', 'location', 'longitude', 'police_district',
       'service_request_number', 'ssa', 'status', 'street_address',
       'type_of_service_request', 'ward',
       'what_type_of_surface_is_the_graffiti_on_', 'Sub Type', 'x_coordinate',
       'y_coordinate', 'zip_code'],
      dtype='object')
Index(['Unnamed: 0', 'address_street_direction', 'address_street_name',
       'address_street_number', 'address_street_suffix',
       'any_people_using_property_homeless_childen_gangs_', 'community_area',
       'creation_date', 'if_the_building_is_open_where_is_the_entry_point_',
       'Sub Type', 'is_the_building_currently_vacant_or_occupied_',
       'is_the_building_vacant_due_to_fire_', 'latitude', 'location',
       'location_of_building_on_the_lot_if_garage_change_type_code_to_bgd_',
       'longitude', 'police_district', 'service_request_number',
       'service_request_type', 'ward', 'x_coordinat

In [7]:
#create sample database to play with
complaints.sample(1000).to_csv('./data/sample.csv')

In [5]:
#number of complaints by type
ctcounts = complaints['Complaint Type'].value_counts()
ctcounts

Graffiti Removal                           116981
Pot Holes Reported                          55885
Sanitation Code Complaints                  21514
Vacant and Abandoned Buildings Reported      4726
Name: Complaint Type, dtype: int64

In [4]:
# GRAPH NUMBER OF REQUESTS BY MONTH AND TYPE
from pandas import TimeGrouper
from matplotlib import pyplot
import numpy as np

n_per_month = complaints[['creation_date', 'Complaint Type', 'service_request_number']].groupby(['creation_date', 'Complaint Type'], as_index=False).count()
n_per_month = n_per_month.pivot(index='creation_date', columns='Complaint Type', values='service_request_number')
n_per_month = n_per_month.set_index(pd.DatetimeIndex(n_per_month.index))

n_per_month = n_per_month[n_per_month.index < '2017-04-01']

groups = n_per_month.groupby(TimeGrouper('M')).aggregate(np.sum)

groups.plot(figsize=(15,10), ylim=(0,15000), title='Number of requests by month and type')
pyplot.show()

In [36]:
complaints.community.value_counts()[:10]

WEST TOWN          12398
LOGAN SQUARE        9629
SOUTH LAWNDALE      6735
BRIGHTON PARK       6615
AVONDALE            6301
LAKE VIEW           6226
LOWER WEST SIDE     5993
IRVING PARK         5845
BELMONT CRAGIN      5098
GAGE PARK           4881
Name: community, dtype: int64

In [4]:
gb = complaints.groupby(['Complaint Type', 'Sub Type']).size().to_frame()
print(gb)

                                                                                         0
Complaint Type                          Sub Type                                          
Graffiti Removal                        Alley                                        35316
                                        Bench                                         2722
                                        Door                                         22303
                                        Dumpster                                     14342
                                        Express Way Job                               1610
                                        Fence                                        16245
                                        Front                                       310521
                                        Garage                                      100033
                                        Garbage Cart                                  6293

In [ ]:
gb_community_area = complaints.groupby(['Complaint Type', 'community_area']).size().to_frame()
select = gb_community_area['0'] 
print(gb_community_area)

### Problem 2

In [13]:
# Adding Block ID to 311 Requests data 
import geopandas as gpd
import pandas as pd
from geopandas import GeoDataFrame
from geopandas.tools import sjoin
from shapely.geometry import Point

#this file contains polygons at the FIPS_12 level (block group)
blocks = gpd.read_file('./data/cb_2015_17_bg_500k/cb_2015_17_bg_500k.shp')

#complaints = pd.read_csv('./data/sample.csv')

clean_data = complaints[complaints['latitude'].notnull() & complaints['longitude'].notnull()]

geometry = [Point(xy) for xy in zip(clean_data.longitude, clean_data.latitude)]
clean_data = clean_data.drop(['latitude', 'longitude'], axis=1)

crs = {'init': 'epsg:4269'}
geo_complaints = GeoDataFrame(clean_data, crs=crs, geometry=geometry)

data_with_blockid = sjoin(geo_complaints, blocks, how="left")

In [14]:
# Getting census data
import requests
import pandas as pd

def get_data_census(varname, label):

    census_api_url = 'http://api.census.gov/data/' + \
                    '2015/acs5?get=NAME,' + varname + \
                    '&for=block+group:*&in=state:17&in=county:031&in=tract:*'

    response = requests.get(census_api_url)

    json_data = response.content

    data = pd.read_json(json_data)

    names = list(data.iloc[0])

    for i in range(0,6):
        data.rename(index=str, columns={i: names[i]}, inplace=True)
    
    data.rename(index=str, columns={varname: label}, inplace=True)
    
    #HERE, I REPLICATE THE FIPS_12 CODE
    data['GEOID'] = data['state'].map(str) + data['county'].map(str) + \
                            data['tract'].map(str) + data['block group'].map(str)

    data.drop(data.index[[0]], inplace=True)
    
    print('downloaded', varname)
    
    return data

download_data = [('B01003_001E', 'Total Population'),
                ('B02001_002E', 'White Population'),
                ('B19013_001E', 'Median Household Income'),
                ('B06009_002E', 'Less than highschool')]

for i, (varname, label) in enumerate(download_data):
    downloaded = get_data_census(varname, label)
    if i == 0:
        census_data = downloaded
    else:
        census_data = census_data.merge(downloaded, on='GEOID', how='outer')

data_with_blockid2 = data_with_blockid.merge(census_data, on='GEOID')    

downloaded B01003_001E
downloaded B02001_002E
downloaded B19013_001E
downloaded B06009_002E


In [26]:
data_with_blockid2.iloc[10]

Unnamed: 0                                                                                                       847629
Complaint Type                                                                                         Graffiti Removal
Sub Type                                                                                                          Front
Unnamed: 0.1                                                                                                      47629
address_street_direction                                                                                            NaN
address_street_name                                                                                                 NaN
address_street_number                                                                                               NaN
address_street_suffix                                                                                               NaN
any_people_using_property_homeless_child

### Problem 3

In [30]:
import requests
from geopy.geocoders import Nominatim
import xml.etree.ElementTree as ET

address = '"7500 S. Wolcott Ave.'
geolocator = Nominatim()
location = geolocator.geocode(address)
print(location.latitude, location.longitude)

census_block_url = 'http://data.fcc.gov/api/block/2010/find?latitude={lat}&longitude={lon}&showall=true'.format(
        lat=location.latitude, lon=location.longitude)

response = requests.get(census_block_url)

xml_data = response.content
etree = ET.fromstring(xml_data)
#print(etree)

for neighbor in etree.iter():
    if 'Block' in neighbor.tag:
        block_id = neighbor.attrib['FIPS'][:12]

data_with_blockid2[data_with_blockid2.GEOID == block_id]["Complaint Type"].value_counts()


41.7573666 -87.6711973381924


Sanitation Code Complaints                 34
Pot Holes Reported                         28
Vacant and Abandoned Buildings Reported    13
Graffiti Removal                           12
Name: Complaint Type, dtype: int64

In [27]:
result = data_with_blockid2[['community', 'Complaint Type', 'service_request_number']].groupby(['community', 'Complaint Type'], as_index=False).count()
select = result[result['Complaint Type'] == 'Graffiti Removal']
select

,community,Complaint Type,service_request_number
0,ALBANY PARK,Graffiti Removal,3658
4,ARCHER HEIGHTS,Graffiti Removal,2691
8,ARMOUR SQUARE,Graffiti Removal,685
12,ASHBURN,Graffiti Removal,1945
16,AUBURN GRESHAM,Graffiti Removal,238
20,AUSTIN,Graffiti Removal,780
24,AVALON PARK,Graffiti Removal,37
28,AVONDALE,Graffiti Removal,5365
32,BELMONT CRAGIN,Graffiti Removal,3605
36,BEVERLY,Graffiti Removal,72


In [25]:
select[select.community.str.contains('LAWNDALE')]

,community,Complaint Type,service_request_number
207,NORTH LAWNDALE,Graffiti Removal,486
254,SOUTH LAWNDALE,Graffiti Removal,5301


In [26]:
select[select.community.str.contains('UPTOWN')]

,community,Complaint Type,service_request_number
262,UPTOWN,Graffiti Removal,1778


In [29]:
print(1778 / 116981)

0.015199049418281601


In [ ]:
len(blocks2.blockce10.value_counts())

In [ ]:
blocks2.columns

In [ ]:
blocks2.countyfp10.value_counts()

In [ ]:
#to make graphs
%matplotlib inline
import matplotlib
blocks.plot();

base = blocks2.plot(color='white')

geo_sel.plot(ax=base, marker='o', color='red', markersize=5);